In [35]:
import pandas as pd
import plotly as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages

In [36]:
def calcola_media_semestrale(df_final):
    periodi_semestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-12-31'))
    ]
    media_semestrale = []
    for start, end in periodi_semestrali:
        media = df_final.loc[start:end].mean()
        media_semestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_semestrale

In [37]:
def calcola_media_trimestrale(df):
    periodi_trimestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_trimestrale = []
    for start, end in periodi_trimestrali:
        media = df.loc[start:end].mean()
        media_trimestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_trimestrale

In [38]:
def calcola_media_bimestrale(df):
    periodi_bimestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-02-28')),
        (pd.Timestamp('2019-03-01'), pd.Timestamp('2019-04-30')),
        (pd.Timestamp('2019-05-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-08-31')),
        (pd.Timestamp('2019-09-01'), pd.Timestamp('2019-10-31')),
        (pd.Timestamp('2019-11-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_bimestrale = []
    for start, end in periodi_bimestrali:
        media = df.loc[start:end].mean()
        media_bimestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_bimestrale

In [39]:
def calcola_media_mensile(df):
    periodi_mensili = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-01-31')),
        (pd.Timestamp('2019-02-01'), pd.Timestamp('2019-02-28')),
        (pd.Timestamp('2019-03-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-04-30')),
        (pd.Timestamp('2019-05-01'), pd.Timestamp('2019-05-31')),
        (pd.Timestamp('2019-06-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-07-31')),
        (pd.Timestamp('2019-08-01'), pd.Timestamp('2019-08-31')),
        (pd.Timestamp('2019-09-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-10-31')),
        (pd.Timestamp('2019-11-01'), pd.Timestamp('2019-11-30')),
        (pd.Timestamp('2019-12-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_mensile = []
    for start, end in periodi_mensili:
        media = df.loc[start:end].mean()
        media_mensile.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_mensile

In [40]:
def calcola_media_breve(valore1, df_dispositivo):
    if valore1=='M' : 
        media_mensile = calcola_media_mensile(df_dispositivo)
        return media_mensile
    # Media per ogni bimestre (2 mesi)
    elif valore1=='B' :
        media_bimestre= calcola_media_bimestrale(df_dispositivo)
        return media_bimestre
    # Media per ogni trimestre (3 mesi)
    elif valore1=='T' :
        media_trimestre = calcola_media_trimestrale(df_dispositivo)
        return media_trimestre

In [41]:
def calcola_media_lunga(valore2,df_dispositivo):
    # Media per ogni semestre (6 mesi)
    if valore2=='S' : 
        media_semestre = calcola_media_semestrale(df_dispositivo)
        return media_semestre
    # Media per ogni bimestre (2 mesi)
    elif valore2=='A' :
        media_annuale = df_dispositivo['consumption'].mean()
        return media_annuale
    # Media per ogni trimestre (3 mesi)
    elif valore2=='T' :
        media_trimestre = calcola_media_trimestrale(df_dispositivo)
        return media_trimestre

In [42]:
def read_fixture(fixture: str, rule='T'):
    #Legge CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (minutale 'T')
    df_minutely = df.resample(rule).sum()
    
    # Ora raggruppiamo i dati minutali in giornate
    df_daily = df_minutely.resample('D').sum()

    # Creare un DataFrame vuoto con il range di date specificato
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31' , freq='D')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_daily, how='left').fillna(0)
    
    return df_final

In [43]:
def read_fixture_sec(fixture: str, rule='S'):
    # Legge il CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (secondi 'S')
    df_secondly = df.resample(rule).sum()
    
    # Creare un DataFrame vuoto con il range di date specificato a livello di secondi
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31 23:59:59', freq='S')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame secondario con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_secondly, how='left').fillna(0)
    
    return df_final

In [44]:
def range_plot_media(df, num):
    #Media delle emissioni degli intervalli di X secondi
    df=df.resample('30S').mean()

    plt.figure(figsize=(10, 6))

    #Grafico per tutte le emissioni
    plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

    #Formatta le date sull'asse x
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

    #Tick per i giorni con colore differenti
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.tick_params(axis='x', which='minor', colors='red')

    # Impostazioni dell'asse y con tick ogni 100 unità
    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
    max_value = df['consumption'].max()
    if max_value>1000:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
    else:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))

    #Impostazioni della griglia
    plt.xlabel('Date')
    plt.ylabel('Consumption (L)')
    #plt.title(f'Emissioni della Settimana {num}')
    plt.legend()
    plt.grid(True, which='major', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', linestyle='-', linewidth=5)
    plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
    plt.xticks(rotation=90)
    plt.show() #X CONTROLLI GRAFIIC

    # Chiudere il grafico per liberare memoria
    plt.close()

In [45]:
def range_plot_media_file(df,weekstart_date,pattern):
    # Media delle emissioni degli intervalli di X secondi
    df = df.resample('30S').mean()

    # Creare una figura e un asse
    fig, ax = plt.subplots(figsize=(10, 6))

    # Grafico per tutte le emissioni
    ax.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

    # Formattare le date sull'asse x
    ax.xaxis.set_major_locator(mdates.SecondLocator(interval=770))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.tick_params(axis='x', which='minor', colors='red',rotation=90)

    # Impostazioni dell'asse y con tick ogni 100 unità
    ax.yaxis.set_major_locator(plt.MultipleLocator(100))
    max_value = df['consumption'].max()
    if max_value > 1000:
        ax.yaxis.set_minor_locator(plt.MultipleLocator(50))
    else:
        ax.yaxis.set_minor_locator(plt.MultipleLocator(20))

    # Impostazioni della griglia
    ax.set_xlabel('Date')
    ax.set_ylabel('Consumption (L)')
    ax.set_title(f'Emissioni di {weekstart_date} : {pattern}')
    ax.legend()
    ax.grid(True, which='major', linestyle='-', linewidth=1)
    ax.grid(True, which='minor', linestyle='-', linewidth=0.5)
    ax.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
    ax.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
    
    # Chiudere il grafico per liberare memoria
    fig.savefig(f'{pattern}.{weekstart_date}.png')
    plt.close(fig)

In [46]:
def range_plot_somma(df, num):
    #Somma degli intervalli di X secondi
    df_sum=df.resample('60S').sum()
    domenica_sum = df_sum[df_sum.index.date == df_sum.index.date[0]]
    week_sum = df_sum[df_sum.index.date > df_sum.index.date[0]]

    plt.figure(figsize=(200, 50))

    #Grafico per tutte le emissioni
    plt.plot(df.index, df['consumption'], label={'consumption': 'Volume [L]'}, color='orange')
    
    #Grafico per le emissioni sommate per certi intervalli
    plt.plot(domenica_sum.index, domenica_sum['consumption'], label={'consumption': 'VolumeSomma [L]'}, color='violet')
    plt.plot(week_sum.index, week_sum['consumption'], label={'consumption': 'VolumeSomma [L]'}, color='blue')

    #Formatta le date sull'asse x
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

    #Tick per i giorni con colore differenti
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.tick_params(axis='x', which='minor', colors='red')

    #Impostazioni della griglia
    plt.xlabel('Date')
    plt.ylabel('Consumption (L)')
    plt.title(f'Emissioni della Settimana {num}')
    plt.legend()
    plt.grid(True, which='major', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', linestyle='-', linewidth=5)
    plt.xticks(rotation=90)
    plt.show()

    # Chiudere il grafico per liberare memoria
    plt.close()

In [47]:
def range_plot(df, num):
    #Somma degli intervalli di X secondi
    #df_sum=df.resample('60S').sum()

    #Media delle emissioni degli intervalli di X secondi
    df_mean=df.resample('30S').mean()
    
    plt.figure(figsize=(100, 50))

    #Grafico per tutte le emissioni
    plt.plot(df.index, df['consumption'], label={'consumption': 'Volume [L]'}, color='red', alpha=0.75, linewidth=0.5)
    
    #Grafico per le emissioni sommate per certi intervalli
    #plt.plot(df_sum.index, df_sum['consumption'], label={'consumption': 'VolumeSomma [L]'}, color='orange')

    #Grafico media per le emissioni per certi intervalli
    plt.plot(df_mean.index, df_mean['consumption'], label={'consumption': 'VolumeMedia[L]'}, color='blue', linewidth=0.5)

    #Formatta le date sull'asse x
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

    #Tick per i giorni con colore differenti
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.tick_params(axis='x', which='minor', colors='red')

    # Impostazioni dell'asse y con tick ogni 100 unità
    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
    max_value = df['consumption'].max()
    if max_value>1000:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
    else:
        plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))
    #Impostazioni della griglia
    plt.xlabel('Date')
    plt.ylabel('Consumption (L)')
    plt.title(f'Emissioni della Settimana {num}')
    plt.legend()
    plt.grid(True, which='major', axis='x', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', axis='x', linestyle='-', linewidth=5)
    plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
    plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
    plt.xticks(rotation=90)
    plt.show()

    # Chiudere il grafico per liberare memoria
    plt.close()

In [48]:
def scelta_pattern_medie_tre(media_bassa, media_med, media_lunga, valore1, valore2, valore3, array_emissioni,num,df_final,df_final_sec,mese_start,giorno_cambio_mese,dizionario_count,dizionario_pattern,weekstart_date):
    #array_emissioni=array dei consumi
    array_pattern = []
    # codice che sceglie pattern     
    prec=0
    val=0

    media_corto = media_medio = media_lungo = 0

    #Media bassa
    if valore1=='M' : 
        media_corto=media_bassa[mese_start-1]
    elif valore1=='B' :
        if mese_start<=2: media_corto=media_bassa[0]
        elif mese_start<=4: media_corto=media_bassa[1]
        elif mese_start<=6: media_corto=media_bassa[2]
        elif mese_start<=8: media_corto=media_bassa[3]
        elif mese_start<=10: media_corto=media_bassa[4]
        else: media_corto=media_bassa[5]
    elif valore1=='T' :
        if mese_start<=3: media_corto=media_bassa[0]
        elif mese_start<=6: media_corto=media_bassa[1]
        elif mese_start<=9: media_corto=media_bassa[2]
        else: media_corto=media_bassa[3]
    
    #Media media
    if valore3=='S' : 
        if mese_start<=6: media_medio=media_med[0]
        else: media_medio=media_med[1]
    elif valore3=='T' :
        if mese_start<=3: media_medio=media_med[0]
        elif mese_start<=6: media_medio=media_med[1]
        elif mese_start<=9: media_medio=media_med[2]
        else: media_medio=media_med[3]
    
    #Media lunga
    if valore2=='S' : 
        if mese_start<=6: media_lungo=media_lunga[0]
        else: media_lungo=media_lunga[1]
    elif valore2=='A' :
        media_lungo=media_lunga

    for i in range(len(array_emissioni) - 1):
        if i==0:
            array_pattern.append("DOM")
            continue
        if giorno_cambio_mese==i:
            if valore1=='M' : 
                media_corto=media_bassa[mese_start]
            elif valore1=='B' :
                if (mese_start+1)<=2: media_corto=media_bassa[0]
                elif (mese_start+1)<=4: media_corto=media_bassa[1]
                elif (mese_start+1)<=6: media_corto=media_bassa[2]
                elif (mese_start+1)<=8: media_corto=media_bassa[3]
                elif (mese_start+1)<=10: media_corto=media_bassa[4]
                else: media_corto=media_bassa[5]
            elif valore1=='T' :
                if (mese_start+1)<=3: media_corto=media_bassa[0]
                elif (mese_start+1)<=6: media_corto=media_bassa[1]
                elif (mese_start+1)<=9: media_corto=media_bassa[2]
                else: media_corto=media_bassa[3]

            if valore2=='S' : 
                if (mese_start+1)<=6: media_lungo=media_lunga[0]
                else: media_lungo=media_lunga[1]
            
            if valore3=='S' : 
                if (mese_start+1)<=6: media_medio=media_med[0]
                else: media_medio=media_med[1]
            elif valore3=='T' :
                if (mese_start+1)<=3: media_medio=media_med[0]
                elif (mese_start+1)<=6: media_medio=media_med[1]
                elif (mese_start+1)<=9: media_medio=media_med[2]
                else: media_medio=media_med[3]
            
        pattern_day=""

        #COMPORTAMENTO GRAFICO
        # Calcola l'intervallo di date
        start_date = weekstart_date + pd.DateOffset(days=i-1)
        end_date = (weekstart_date + pd.DateOffset(days=i-1)).replace(hour=23, minute=59, second=59)
        # Seleziona i dati
        df_slice = df_final_sec[start_date:end_date]
        #pattern_day+=pattern_comportamento(df_slice)

        #PATTERN MEDIA CORTO
        if array_emissioni[i]<(media_corto/4):
            pattern_day+= "CA"
        elif array_emissioni[i]<(media_corto/2):
            pattern_day+= "CB"
        elif array_emissioni[i]<(media_corto*(3/4)):
            pattern_day+= "CC"
        elif array_emissioni[i]<media_corto:
            pattern_day+= "CD"
        elif array_emissioni[i]<(media_corto*(5/4)):
            pattern_day+= "CE"
        elif array_emissioni[i]<(media_corto*(3/2)):
            pattern_day+= "CF"
        elif array_emissioni[i]<(media_corto*(7/4)):
            pattern_day+= "CG"
        else:
            pattern_day+= "CH"
        
        #PATTERN MEDIA MEDIA
        if array_emissioni[i]<(media_medio/4):
            pattern_day+= "MA"
        elif array_emissioni[i]<(media_medio/2):
            pattern_day+= "MB"
        elif array_emissioni[i]<(media_medio*(3/4)):
            pattern_day+= "MC"
        elif array_emissioni[i]<media_medio:
            pattern_day+= "MD"
        elif array_emissioni[i]<(media_medio*(5/4)):
            pattern_day+= "ME"
        elif array_emissioni[i]<(media_medio*(3/2)):
            pattern_day+= "MF"
        elif array_emissioni[i]<(media_medio*(7/4)):
            pattern_day+= "MG"
        else:
            pattern_day+= "MH"

        #PATTERN MEDIA LUNGO
        if array_emissioni[i]<(media_lungo / 4):
            pattern_day += "LA"
        elif array_emissioni[i]<(media_lungo / 2):
            pattern_day += "LB"
        elif array_emissioni[i]<(media_lungo * (3 / 4)):
            pattern_day += "LC"
        elif array_emissioni[i]<media_lungo:
            pattern_day += "LD"
        elif array_emissioni[i]<(media_lungo * (5 / 4)):
            pattern_day += "LE"
        elif array_emissioni[i]<(media_lungo * (3 / 2)):
            pattern_day += "LF"
        elif array_emissioni[i]<(media_lungo * (7 / 4)):
            pattern_day += "LG"
        else:
            pattern_day += "LH"

        #CALCOLARE RAPPORTO TRA LE DUE EMISSIONI
        if i>0:
            if array_emissioni[i-1] != 0:
                val = array_emissioni[i] / array_emissioni[i-1]
            else:
                val = array_emissioni[i]
            prec=array_emissioni[i-1]
        else: val = array_emissioni[i]

        #PATTERN RAPPORTO
        if val == 0:  # zero emissioni
            if prec == 0:#solitamente il LUN
                pattern_day+= "0A"
            else:
                pattern_day+= "0B"

        elif val <= 1:  # emissioni diminuite
            if val < 0.20:
                pattern_day+= "1A"
            elif val < 0.50:
                pattern_day+= "1B"
            elif val < 0.75:
                pattern_day+= "1C"
            else:
                pattern_day+= "1D"

        elif val <= 2:  # emissioni aumentate leggermente
            if val < 1.20:
                pattern_day+= "2A"
            elif val < 1.50:
                pattern_day+= "2B"
            elif val < 1.75:
                pattern_day+= "2C"
            else:
                pattern_day+= "2D"

        elif val <= 10:  # emissioni aumentate
            if val < 7:
                pattern_day+= "3A"
            else: pattern_day+= "3B" 

        else:
            if i>0: pattern_day+= "4B" #valore un po strano
            else: pattern_day+= "4A" # possibile, probabilmente LUN
        array_pattern.append(pattern_day)
        
    # Stampa l'array pattern e emissioni
    print(f"\nArray con i pattern della settimana {num} da LUN a DOM")
    print(array_pattern)
    #print(f"\nArray con le emissioni della settimana {num} da LUN a DOM")
    #print(array_emissioni)
    
    #Calcola presenze dei pattern
    for pat in array_pattern:
        if pat in dizionario_count:#se lo trova 
            dizionario_count[pat] += 1
        else:
            dizionario_count[pat] = 1
    
    #Calcola giorni dei pattern
    for i in range(len(array_pattern)):
        if array_pattern[i]=="DOM":#skip della prima dom
            continue
        # Calcola l'intervallo di date
        start_date = weekstart_date + pd.DateOffset(days=i-1)
        end_date = (weekstart_date + pd.DateOffset(days=i-1)).replace(hour=23, minute=59, second=59)
        # Seleziona i dati
        df_slice = df_final_sec[start_date:end_date]
        # Aggiorna dizionario grafici
        if array_pattern[i] in dizionario_pattern:#se lo trova
            dizionario_pattern[array_pattern[i]].append(df_slice)#weekstart_date + pd.DateOffset(days=i-1)
        else:
            dizionario_pattern[array_pattern[i]]=[]
            dizionario_pattern[array_pattern[i]].append(df_slice)
            #range_plot_media_return(df_slice,weekstart_date,array_pattern[i])
    
    #Stampa il grafico solo se abbiamo delle emissioni durante gli 8 giorni
    #for i in range(len(array_emissioni) - 1):
    #    if array_emissioni[i] !=0:
    #        print(f"\nGrafico delle emissioni della settimana {num}") 
    #        #range_plot(df_final_sec, num) #Grafico per verifica comportamento dispositivo
    #        #range_plot_somma(df_final_sec, num)
    #        range_plot_media(df_final_sec, num) #Grafico principale 
    #        break

In [ ]:
def ciclo_pattern_settimane_precise_medie_tre(df_dispositivo,df_dispositivo_sec,media_bassa, media_med, media_lunga, valore1, valore2, valore3):
    dizionario_count = {}
    dizionario_pattern={}
    num=0 #numero settimana
    start_date = pd.Timestamp('2019-01-01')
    end_date = pd.Timestamp('2019-12-31')
    #trovo il prossimo lunedi
    start_date += pd.DateOffset(days=(7 - start_date.weekday())) #0=lunedi 6=domenica
    current_date=start_date
    while current_date <= end_date:
        weekstart_date=current_date
        weekend_date=weekstart_date + pd.DateOffset(days=6)
        if weekend_date > end_date:
            break
        # Creare un range di date
        date_rng = pd.date_range(start= (weekstart_date-pd.DateOffset(days=1)), end=weekend_date, freq='D')
        current_date += pd.DateOffset(weeks=1)

        # Creare un DataFrame vuoto con il range di date specificato
        df_final = pd.DataFrame(date_rng, columns=['date'])
        df_final.set_index('date', inplace=True)
        
        # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
        df_final = df_final.join(df_dispositivo, how='left').fillna(0)
        
        array_emissioni = df_final['consumption'].to_numpy()

        # Controlliamo se ci troviamo in una settimana tra due mesi, causa controlli con le medie
        mese_start = weekstart_date.month
        giorno_cambio_mese = -1
        for i in range(7):
            current_day = weekstart_date + pd.DateOffset(days=i)
            if current_day.month != mese_start:
                giorno_cambio_mese = i #gia sappiamo che se cambia mese aumenta mese_start di 1
                break
        
        #campionamento per secondi
        df_final_sec=df_dispositivo_sec[(weekstart_date-pd.DateOffset(days=1)): (weekend_date.replace(hour=23, minute=59, second=59))]
        scelta_pattern_medie_tre(media_bassa, media_med, media_lunga, valore1, valore2, valore3, array_emissioni,num,df_final,df_final_sec,mese_start,giorno_cambio_mese,dizionario_count,dizionario_pattern,weekstart_date)
        num=num+1
        #per velocizzare i test mi fermo alla settimana numero 7
        #if num==7:
        #    break
    print(dizionario_count)
    # DEBUG CONTO GRAFICI
    # print(dizionario_pattern)
    #for key,value in dizionario_pattern.items():
    #    count=0
    #    for plot in value:
    #        count+=1
    #        # Stampa l'oggetto Figure range_plot_media(plot, 0)
    #    print(f'\n{key}=={count}')
    print(f'Numero di chiavi nel dizionario: {len(dizionario_pattern)}')
    pdf_counter = 0
    
    for key,value in dizionario_pattern.items():
        print(f'\n{key}')
        pdf_filename = f'plotgrafico\{key}.pdf'
        with PdfPages(pdf_filename) as pdf:
            for graf in value:
                # Stampa l'oggetto Figure 
                if not isinstance(graf, pd.DataFrame) or 'consumption' not in graf.columns:#CONTROLLO 
                    print(f'Warning: Invalid DataFrame for key {key}')
                    continue

                #Media delle emissioni degli intervalli di X secondi
                df=graf.resample('30S').mean()

                plt.figure(figsize=(10, 6))

                #Grafico per tutte le emissioni
                plt.plot(df.index, df['consumption'], label={'consumption': 'VolumeMedia [L]'}, color='red', linewidth=0.5)

                #Formatta le date sull'asse x
                plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=770))

                #Tick per i giorni con colore differenti
                plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%m-%d'))
                plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
                plt.tick_params(axis='x', which='minor', colors='red')

                # Impostazioni dell'asse y con tick ogni 100 unità
                plt.gca().yaxis.set_major_locator(plt.MultipleLocator(100))
                max_value = df['consumption'].max()
                if max_value>1000:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(50))
                else:
                    plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(20))

                #Impostazioni della griglia
                plt.xlabel('Date')
                plt.ylabel('Consumption (L)')
                plt.title(f'Emissioni del pattern {key}')
                plt.legend()
                plt.grid(True, which='major', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', linestyle='-', linewidth=5)
                plt.grid(True, which='major', axis='y', linestyle='-', linewidth=1)
                plt.grid(True, which='minor', axis='y', linestyle='-', linewidth=1)
                plt.xticks(rotation=90)

                pdf.savefig() #X CONTROLLI GRAFIIC

                # Chiudere il grafico per liberare memoria
                plt.close()
        pdf_counter += 1
        #print(f'Fine per {key}')
    print(f'Total PDFs created: {pdf_counter}')


In [50]:
#Nome dispositivo, tipi medie: Mensile(M),Bimestre(B),Trimestre(T),Semestre(S),Annuale(A) 
#TO DO Scelta della media direttamente da qua, inserendo due medie o tre 
#Combinazioni Possibili
print("Combinazioni possibili: MTA, BSA, MTS, TSA")
df_dispositivo=read_fixture('Washbasin')
df_dispositivo_sec=read_fixture_sec('Washbasin')
#Scelta medie
valore1 = 'M' #input("Media bassa [M] [B] [T]")
valore2 = 'T' #input("Media med [T] [S]")
valore3 = 'S' #input("Media lunga [S] [A]")

media_bassa=calcola_media_breve(valore1,df_dispositivo)
media_med=calcola_media_lunga(valore2,df_dispositivo)
media_lunga=calcola_media_lunga(valore3,df_dispositivo)

#debug print_medie_tre(media_bassa, media_med, media_lunga,valore1,valore2,valore3)

ciclo_pattern_settimane_precise_medie_tre(df_dispositivo,df_dispositivo_sec,media_bassa, media_med, media_lunga, valore1, valore2, valore3)

Combinazioni possibili: MTA, BSA, MTS, TSA

Array con i pattern della settimana 0 da LUN a DOM
['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

Array con i pattern della settimana 1 da LUN a DOM
['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

Array con i pattern della settimana 2 da LUN a DOM
['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

Array con i pattern della settimana 3 da LUN a DOM
['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A']

Array con i pattern della settimana 4 da LUN a DOM
['DOM', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A']

Array con i pattern della settimana 5 da LUN a DOM
['DOM', 'CAMALH0A', 'CAMALH0A', 'CAMALH4B', 'CAMALH0B', 'CAMALH0A', 'CCMDLH4B']

Array con i pattern della settimana 6 da LUN a DOM
['DOM', 'CGMHLH1C', 'CHMHLH2B', 'CHMHLH2D', 'CFMHLH1B', 'CHMHLH2D', 'CGMHLH1C']

Array con i pattern della settim

In [51]:
from collections import Counter

#CODICE PER OTTENERE LE VOLTE CHE UN PATTERN SI PRESENTA
# Liste dei pattern per ogni settimana
patterns = [
     ['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A']

,['DOM', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A', 'CAMALH0A']

,['DOM', 'CAMALH0A', 'CAMALH0A', 'CAMALH4B', 'CAMALH0B', 'CAMALH0A', 'CCMDLH4B']

,['DOM', 'CGMHLH1C', 'CHMHLH2B', 'CHMHLH2D', 'CFMHLH1B', 'CHMHLH2D', 'CGMHLH1C']

,['DOM', 'CHMHLH2C', 'CHMHLH2B', 'CGMHLH1B', 'CHMHLH2D', 'CEMHLH1C', 'CDMGLH1D']

,['DOM', 'CFMHLH1D', 'CDMGLH1C', 'CCMDLH1C', 'CCMDLH2A', 'CAMALH0B', 'CAMALH0A']

,['DOM', 'CFMHLH2C', 'CAMBLH1A', 'CEMGLH3A', 'CDMFLH1C', 'CEMHLH2B', 'CFMHLH2B']

,['DOM', 'CAMALH1A', 'CDMGLH3B', 'CDMGLH2A', 'CDMGLH1D', 'CDMGLH1D', 'CEMHLH2B']

,['DOM', 'CDMFLH1A', 'CHMHLH3A', 'CCMDLH1B', 'CFMHLH3A', 'CCMDLH1B', 'CCMDLH1D']

,['DOM', 'CHMHLH3B', 'CBMELH1A', 'CCMFLH2B', 'CDMHLH2B', 'CEMHLH2B', 'CDMHLH1C']

,['DOM', 'CAMALH0B', 'CBMDLH4B', 'CCMELH2C', 'CBMDLH1D', 'CEMHLH3A', 'CFMHLH2A']

,['DOM', 'CEMHLH3B', 'CDMHLH1D', 'CEMHLH2B', 'CHMHLH3A', 'CBMCLH1A', 'CDMHLH3A']

,['DOM', 'CFMHLH4B', 'CHMHLH3A', 'CAMALH1A', 'CAMALH2A', 'CDMHLH3B', 'CHMHLH3A']

,['DOM', 'CCMFLH1D', 'CAMALH1A', 'CHMHLH4B', 'CCMELH1A', 'CDMGLH2B', 'CFMHLH2C']

,['DOM', 'CDMHLH2B', 'CDMFLH1D', 'CEMHLH2B', 'CCMFLH1C', 'CCMFLH2A', 'CAMALH0B']

,['DOM', 'CAMALH4B', 'CDMHLH4B', 'CEMHLH2B', 'CDMGLH1C', 'CDMGLH2A', 'CHMHLH3A']

,['DOM', 'CBMDLH1C', 'CGMHLH3A', 'CFMHLH1D', 'CGMHLH2A', 'CDMGLH1C', 'CFMHLH2C']

,['DOM', 'CAMALH4B', 'CDMHLH3B', 'CBMDLH1B', 'CHMHLH3A', 'CAMALH0B', 'CAMALH4B']

,['DOM', 'CDMHLH2A', 'CDMHLH2A', 'CBMGLH1B', 'CCMHLH2B', 'CCMHLH1D', 'CBMGLH1C']

,['DOM', 'CFMHLH3A', 'CBMDLH1A', 'CEMHLH3A', 'CEMHLH2A', 'CHMHLH2D', 'CHMHLH3A']

,['DOM', 'CCMHLH2C', 'CCMHLH1D', 'CHMHLH3A', 'CBMDLH1A', 'CEMHLH3A', 'CEMHLH2A']

,['DOM', 'CCMHLH2C', 'CEMHLH2C', 'CCMHLH1B', 'CAMALH1A', 'CAMALH0B', 'CAMALH4B']

,['DOM', 'CEMCLH1C', 'CAMALH1B', 'CEMCLH3A', 'CCMBLH1C', 'CAMALH1A', 'CHMELH4B']

,['DOM', 'CFMDLH2B', 'CDMBLH1C', 'CGMDLH3A', 'CCMBLH1B', 'CHMHLH3A', 'CEMCLH1B']

,['DOM', 'CDMBLH1B', 'CHMELH3A', 'CCMBLH1B', 'CCMBLH2A', 'CAMALH1A', 'CAMALH0B']

,['DOM', 'CDMCLH2B', 'CDMCLH1D', 'CDMBLH1D', 'CAMALH1A', 'CCMBLH3A', 'CEMCLH2C']

,['DOM', 'CDMCLH1D', 'CFMDLH2C', 'CHMCLH1C', 'CAMALH0B', 'CAMALH0A', 'CAMALH0A']

,['DOM', 'CHMGLH3A', 'CDMBLH1A', 'CDMALH1D', 'CCMALH1C', 'CGMBLH3A', 'CAMALH0B']

,['DOM', 'CAMALH0A', 'CDMALH4B', 'CAMALH1A', 'CFMBLH4B', 'CFMBLH2A', 'CCMALH1B']

,['DOM', 'CHMDLH3A', 'CAMALH0B', 'CAMALH0A', 'CDMALH4B', 'CAMALH0B', 'CAMALH0A']

,['DOM', 'CAMALH0B', 'CAMALH0A', 'CAMALH0A', 'CAMALH4B', 'CEMBLH4B', 'CGMELH3A']

,['DOM', 'CDMCLH4B', 'CBMBLH1C', 'CEMDLH3A', 'CDMCLH1D', 'CBMBLH1B', 'CCMBLH2A']

,['DOM', 'CCMBLH4B', 'CGMELH3A', 'CBMBLH1B', 'CAMALH0B', 'CCMBLH4B', 'CGMELH3A']

,['DOM', 'CCMBLH3A', 'CGMELH3A', 'CEMDLH1C', 'CEMDLH1D', 'CDMCLH1C', 'CGMELH3A']

,['DOM', 'CHMHLH3A', 'CHMFLH1C', 'CFMDLH1C', 'CCMCLH1C', 'CHMFLH3A', 'CGMELH1D']

,['DOM', 'CDMDLH1C', 'CEMELH2B', 'CAMALH1A', 'CBMBLH4B', 'CGMGLH3A', 'CDMDLH1C']

,['DOM', 'CBMBLH1B', 'CHMHLH3A', 'CEMELH1C', 'CHMHLH2D', 'CBMBLH1B', 'CHMHLH3A']

,['DOM', 'CHMHLH2B', 'CCMCLH1B', 'CCMCLH1C', 'CCMDLH2B', 'CCMDLH1D', 'CHMHLH3A']

,['DOM', 'CBMBLH2B', 'CEMFLH3A', 'CBMBLH1B', 'CCMCLH2B', 'CDMDLH2B', 'CEMELH2B']

,['DOM', 'CFMFLH1D', 'CAMALH0B', 'CAMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']

,['DOM', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A', 'CHMALH0A']  
]

# Combina tutte le liste in un'unica lista
all_patterns = [pattern for week in patterns for pattern in week]

# Conta le occorrenze di ogni pattern
pattern_counts = Counter(all_patterns)

# Mostra i risultati
print(f'Numero di chiavi nel dizionario: {len(pattern_counts)}')

for pattern, count in pattern_counts.items():
    print(f"{pattern}: {count}")


Numero di chiavi nel dizionario: 128
DOM: 51
CHMALH0A: 72
CAMALH0A: 21
CAMALH4B: 6
CAMALH0B: 14
CCMDLH4B: 1
CGMHLH1C: 2
CHMHLH2B: 3
CHMHLH2D: 5
CFMHLH1B: 1
CHMHLH2C: 1
CGMHLH1B: 1
CEMHLH1C: 1
CDMGLH1D: 3
CFMHLH1D: 2
CDMGLH1C: 3
CCMDLH1C: 1
CCMDLH2A: 1
CFMHLH2C: 3
CAMBLH1A: 1
CEMGLH3A: 1
CDMFLH1C: 1
CEMHLH2B: 6
CFMHLH2B: 1
CAMALH1A: 9
CDMGLH3B: 1
CDMGLH2A: 2
CDMFLH1A: 1
CHMHLH3A: 13
CCMDLH1B: 2
CFMHLH3A: 2
CCMDLH1D: 2
CHMHLH3B: 1
CBMELH1A: 1
CCMFLH2B: 1
CDMHLH2B: 2
CDMHLH1C: 1
CBMDLH4B: 1
CCMELH2C: 1
CBMDLH1D: 1
CEMHLH3A: 3
CFMHLH2A: 1
CEMHLH3B: 1
CDMHLH1D: 1
CBMCLH1A: 1
CDMHLH3A: 1
CFMHLH4B: 1
CAMALH2A: 1
CDMHLH3B: 2
CCMFLH1D: 1
CHMHLH4B: 1
CCMELH1A: 1
CDMGLH2B: 1
CDMFLH1D: 1
CCMFLH1C: 1
CCMFLH2A: 1
CDMHLH4B: 1
CBMDLH1C: 1
CGMHLH3A: 1
CGMHLH2A: 1
CBMDLH1B: 1
CDMHLH2A: 2
CBMGLH1B: 1
CCMHLH2B: 1
CCMHLH1D: 2
CBMGLH1C: 1
CBMDLH1A: 2
CEMHLH2A: 2
CCMHLH2C: 2
CEMHLH2C: 1
CCMHLH1B: 1
CEMCLH1C: 1
CAMALH1B: 1
CEMCLH3A: 1
CCMBLH1C: 1
CHMELH4B: 1
CFMDLH2B: 1
CDMBLH1C: 1
CGMDLH3A: 1
CCMBLH1B: 2
CEM

In [52]:
#CODICE PER DEBUG, PROBLEMI CON SALVATAGGIO FILE PDF
listatot=[
    "CDmALD0A",
    "cAmALD0A",
    "cAmALD4B",
    "cAmALD0B",
    "cCmDLD4B",
    "CCMDLD1C",
    "CDMDLD2B",
    "CDMDLD2D",
    "CBMDLD1B",
    "CDMDLD2C",
    "CCMDLD1B",
    "CAMDLD1C",
    "cDMCLD1D",
    "CBMDLD1D",
    "cDMCLD1C",
    "cCmDLD1C",
    "cCmDLD2A",
    "CBMDLD2C",
    "cAmBLD1A",
    "CAMCLD3A",
    "cDMBLD1C",
    "CAMDLD2B",
    "CBMDLD2B",
    "cAmALD1A",
    "cDMCLD3B",
    "cDMCLD2A",
    "cDMBLD1A",
    "CDMDLD3A",
    "cCmDLD1B",
    "CBMDLD3A",
    "cCmDLD1D",
    "CDMDLD3B",
    "cBMALD1A",
    "cCMBLD2B",
    "cDMDLD2B",
    "cDMDLD1C",
    "cBmDLD4B",
    "cCMALD2C",
    "cBmDLD1D",
    "CAMDLD3A",
    "CBMDLD2A",
    "CAMDLD3B",
    "cDMDLD1D",
    "cBmCLD1A",
    "cDMDLD3A",
    "CBMDLD4B",
    "cAmALD2A",
    "cDMDLD3B",
    "cCMBLD1D",
    "CDMDLD4B",
    "cCMALD1A",
    "cDMCLD2B",
    "cDMBLD1D",
    "cCMBLD1C",
    "cCMBLD2A",
    "cDMDLD4B",
    "cBmDLD1C",
    "CCMDLD3A",
    "CCMDLD2A",
    "cBmDLD1B",
    "cDMDLD2A",
    "cBMCLD1B",
    "cCMDLD2B",
    "cCMDLD1D",
    "cBMCLD1C",
    "cBmDLD1A",
    "CAMDLD2A",
    "cCMDLD2C",
    "CAMDLD2C",
    "cCMDLD1B",
    "CAmCLD1C",
    "cAmALD1B",
    "CAmCLD3A",
    "cCmBLD1C",
    "CDMALD4B",
    "CBmDLD2B",
    "cDmBLD1C",
    "CCmDLD3A",
    "cCmBLD1B",
    "CAmCLD1B",
    "cDmBLD1B",
    "CDMALD3A",
    "cCmBLD2A",
    "cDmCLD2B",
    "cDmCLD1D",
    "cDmBLD1D",
    "cCmBLD3A",
    "CAmCLD2C",
    "CBmDLD2C",
    "CDmCLD1C",
    "CDMCLD3A",
    "cDmBLD1A",
    "cDmALD1D",
    "cCmALD1C",
    "CCmBLD3A",
    "cDmALD4B",
    "CBmBLD4B",
    "CBmBLD2A",
    "cCmALD1B",
    "CDmDLD3A",
    "CAmBLD4B",
    "CCMALD3A",
    "cDmCLD4B",
    "cBmBLD1C",
    "CAmDLD3A",
    "cBmBLD1B",
    "cCmBLD4B",
    "CAmDLD1C",
    "CAmDLD1D",
    "cDmCLD1C",
    "CDMBLD1C",
    "CBmDLD1C",
    "cCmCLD1C",
    "CDMBLD3A",
    "CCMALD1D",
    "cDmDLD1C",
    "CAMALD2B",
    "cBmBLD4B",
    "CCMCLD3A",
    "CAMALD1C",
    "cCmCLD1B",
    "cCmDLD2B",
    "cBmBLD2B",
    "CAMBLD3A",
    "cCmCLD2B",
    "cDmDLD2B",
    "CBMBLD1D"
]

listafile=[
    "cAmALD0A",
    "cAmALD0B",
    "cAmALD1A",
    "cAmALD1B",
    "CAMALD1C",
    "cAmALD2A",
    "CAMALD2B",
    "cAmALD4B",
    "cAmBLD1A",
    "CAMBLD3A",
    "CAmBLD4B",
    "CAmCLD1B",
    "CAmCLD1C",
    "CAmCLD2C",
    "CAMCLD3A",
    "CAMDLD1C",
    "CAmDLD1D",
    "CAMDLD2A",
    "CAMDLD2B",
    "CAMDLD2C",
    "CAMDLD3A",
    "CAMDLD3B",
    "cBMALD1A",
    "cBmBLD1B",
    "cBmBLD1C",
    "CBMBLD1D",
    "cBmBLD2B",
    "CBmBLD2A",
    "CBmBLD4B",
    "cBmCLD1A",
    "cBMCLD1B",
    "cBMCLD1C",
    "cBmDLD1A",
    "CBMDLD1B",
    "cBmDLD1C",
    "CBMDLD1D",
    "CBMDLD2A",
    "CBMDLD2B",
    "CBMDLD2C",
    "CBMDLD3A",
    "cBmDLD4B",
    "cCMALD1A",
    "cCmALD1B",
    "cCmALD1C",
    "CCMALD1D",
    "cCMALD2C",
    "CCMALD3A",
    "cCmBLD1B",
    "cCMBLD1C",
    "cCMBLD1D",
    "cCMBLD2A",
    "cCMBLD2B",
    "cCmBLD3A",
    "cCmBLD4B",
    "cCmCLD1B",
    "cCmCLD1C",
    "cCmCLD2B",
    "CCMCLD3A",
    "CCMDLD1B",
    "CCMDLD1C",
    "cCmDLD1D",
    "cCmDLD2A",
    "cCMDLD2B",
    "cCMDLD2C",
    "CCMDLD3A",
    "cCmDLD4B",
    "CDmALD0A",
    "cDmALD1D",
    "CDMALD3A",
    "CDMALD4B",
    "cDMBLD1A",
    "cDmBLD1B",
    "cDMBLD1C",
    "cDMBLD1D",
    "CDMBLD3A",
    "cDMCLD1C",
    "cDMCLD1D",
    "cDMCLD2A",
    "cDMCLD2B",
    "CDMCLD3A",
    "cDMCLD3B",
    "cDmCLD4B",
    "cDMDLD1C",
    "cDMDLD1D",
    "cDMDLD2A",
    "CDMDLD2B",
    "CDMDLD2C",
    "CDMDLD2D",
    "CDMDLD3A",
    "CDMDLD3B",
    "CDMDLD4B"
]

listanontrovati=[
    "cCmDLD1C",
    "cCmDLD1B",
    "cDMDLD2B",
    "cBmDLD1D",
    "cDMDLD3A",
    "CBMDLD4B",
    "cDMDLD3B",
    "cDMDLD4B",
    "CCMDLD2A",
    "cBmDLD1B",
    "cCMDLD1D",
    "cCMDLD1B",
    "CAmCLD3A",
    "cCmBLD1C",
    "CBmDLD2B",
    "cDmBLD1C",
    "CCmDLD3A",
    "cCmBLD2A",
    "cDmCLD2B",
    "cDmCLD1D",
    "cDmBLD1D",
    "CBmDLD2C",
    "CDmCLD1C",
    "cDmBLD1A",
    "CCmBLD3A",
    "cDmALD4B",
    "CDmDLD3A",
    "CAmDLD3A",
    "CAmDLD1C",
    "cDmCLD1C",
    "CDMBLD1C",
    "CBmDLD1C",
    "cDmDLD1C",
    "cBmBLD4B",
    "cCmDLD2B",
    "cDmDLD2B",
]

print(f'PATTERN SENZA FILE, tot={len(listatot)} file={len(listafile)} differenza{len(listatot)-len(listafile)}\n')
totcont=0
for elem in listatot:
    cont=0
    for elemfile in listafile:
        if elem==elemfile:
            cont+=1
            break
    if cont==0: 
        totcont+=1
        print(f'{elem}:{cont}\n')
print(f'PATTERN trovati={totcont}' )

#DEBUG 
totcont=0
for elem in listanontrovati:
    cont=0
    for elemfile in listafile:
        if elem==elemfile:
            print(f'{elem}\n')
            totcont+=1
            break
print(f'PATTERN trovati={totcont} che non dovevo trovare\n'  )

PATTERN SENZA FILE, tot=127 file=91 differenza36

cCmDLD1C:0

cCmDLD1B:0

cDMDLD2B:0

cBmDLD1D:0

cDMDLD3A:0

CBMDLD4B:0

cDMDLD3B:0

cDMDLD4B:0

CCMDLD2A:0

cBmDLD1B:0

cCMDLD1D:0

cCMDLD1B:0

CAmCLD3A:0

cCmBLD1C:0

CBmDLD2B:0

cDmBLD1C:0

CCmDLD3A:0

cCmBLD2A:0

cDmCLD2B:0

cDmCLD1D:0

cDmBLD1D:0

CBmDLD2C:0

CDmCLD1C:0

cDmBLD1A:0

CCmBLD3A:0

cDmALD4B:0

CDmDLD3A:0

CAmDLD3A:0

CAmDLD1C:0

cDmCLD1C:0

CDMBLD1C:0

CBmDLD1C:0

cDmDLD1C:0

cBmBLD4B:0

cCmDLD2B:0

cDmDLD2B:0

PATTERN trovati=36
PATTERN trovati=0 che non dovevo trovare

